In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random as rd
from datetime import datetime
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
training_labels = pd.read_csv("set_entrenamiento.csv", low_memory=False)[['person','label']].head(1960)
training_labels['label'].value_counts()

1    980
0    980
Name: label, dtype: int64

In [4]:
def elegir_columnas_random(columnas,n):
    rd.seed(n)
    lista_columnas = columnas.copy()
    cantidad = rd.randint(4, len(lista_columnas))
    columnas_prohibidas = ['person','label','checkout']
    for x in range(cantidad):
        c = rd.choice(lista_columnas)
        if c in columnas_prohibidas:
            continue
        lista_columnas.remove(c)
    return lista_columnas

In [5]:
def grid_search():
    n_estimators = [20,30,40,50,60,70]
    min_samples_split = [10,20,30,40,50]
    n_jobs = [-5,-1]
    maximos = []
    max_score = 0
    for n_est in n_estimators:
        for min_sam in min_samples_split:
            for n_j in n_jobs:
                c=RandomForestClassifier(n_estimators=n_est,min_samples_split=min_sam,n_jobs=n_j,random_state=0,class_weight="balanced",criterion = 'entropy')
                dt=c.fit(x_train,y_train)
                score=c.score(x_test,y_test)*100
                if max_score < score:
                    maximos = [n_est,min_sam,n_j]
                    max_score = score
                #print("n_estimators : "+ str(n_est)+"; min_samples_split: "+str(min_sam)+"; n_job: "+str(n_j)+"; SCORE: "+ str(score))
    #print("El mejor score fue: "+str(max_score))
    return maximos

In [6]:
def actualizar_score(score,arbol_score,mejor_arbol_score,cols):
    if score > arbol_score:
        arbol_score = score
    if score > mejor_arbol_score:
        columnas_arbol_mejor_score = cols.copy()
        mejor_arbol_score = score
        sem = x
    return score,arbol_score,mejor_arbol_score,cols

In [7]:
pd.read_csv("data_set.csv", low_memory = False).columns

Index(['person', 'Email', 'Organic', 'Paid', 'Referral', 'Social', 'Unknown',
       'ad campaign hit', 'brand listing', 'checkout', 'conversion',
       'generic listing', 'lead', 'search engine hit', 'searched products',
       'staticpage', 'viewed product', 'visited site', 'promedio dias',
       'promedio hora', 'retornos', 'longitud busqueda', 'cantidad mayusculas',
       'coincide', 'sessions', 'total_time', 'mean_time_by_session',
       'max_session_time', 'mean_events_by_session', 'max_events_by_session',
       'days_since_last_session', 'promedio de TFIdf',
       'different_products_views', 'total_views', 'same_product_max_views'],
      dtype='object')

In [8]:
columnas = ['person', 'Email', 'Organic', 'Paid', 'Referral', 'Social',
       'Unknown', 'ad campaign hit', 'brand listing', 'checkout',
       'conversion', 'generic listing', 'lead', 'search engine hit',
       'searched products', 'staticpage', 'viewed product',
       'visited site', 'promedio dias', 'promedio hora', 'retornos',
       'longitud busqueda', 'cantidad mayusculas', 'coincide', 'sessions',
       'total_time', 'mean_time_by_session', 'max_session_time',
       'mean_events_by_session', 'max_events_by_session',
       'days_since_last_session', 'promedio de TFIdf',
       'different_products_views', 'total_views',
       'same_product_max_views']  #Deben ser columnas del csv que se pasa con los features

In [9]:
RANDOM_SEED = 123
predicciones = pd.read_csv("trocafone_kaggle_test.csv", low_memory=False)
entrenamiento = pd.read_csv("set_entrenamiento.csv", low_memory=False).head(1960)
entrenamiento = entrenamiento[['person','label']]
mejor_arbol_score = 0
columnas_arbol_mejor_score = []
sem = 0
arbol_score = 0
for x in range(100):
    print("Arbol : "+str(x))
    
    #Preparo los dataframe
    training_labels = pd.read_csv("set_entrenamiento.csv", low_memory=False).head(1960)#la idea es pasar un set que sean mitad 0 y mitad 1 sus labels
    labels_predict = pd.read_csv("trocafone_kaggle_test.csv", low_memory=False)
    data = pd.read_csv("data_set.csv", low_memory = False)
    training_labels = training_labels[['person','label']]
    cols = elegir_columnas_random(columnas,x)
    data = data[cols]
    features = pd.merge(training_labels, data, on='person', how='inner')
    features = features.fillna(0)
    
    #Acomodo los sets de test y entrenamiento
    train, test = train_test_split(features,test_size=0.30,random_state=RANDOM_SEED)
    features=list(features.columns)
    features.remove('person')
    features.remove('label')
    x_train=train[features]
    y_train=train['label']
    x_test=test[features]
    y_test=test['label']
    
    
    #Entreno arbol con los mejores hiperparametros encontrados
    mejores_hiperparmetros = grid_search()
    n_est= mejores_hiperparmetros[0]
    min_sam = mejores_hiperparmetros[1]
    n_j= mejores_hiperparmetros[2]
    c=RandomForestClassifier(n_estimators=n_est,min_samples_split=min_sam,n_jobs=n_j,random_state=0,class_weight="balanced",criterion = 'entropy')
    dt=c.fit(x_train,y_train)
    
    #test para knn
    prediccion_test = c.predict(x_test)
    if x == 0:
        entrenamiento = pd.merge(test[['person','label']],entrenamiento,on='person',how='left')
    entrenamiento['pred'+str(x)] = prediccion_test
    
    #Veo los resultados
    score=c.score(x_test,y_test)*100
    score,arbol_score,mejor_arbol_score,cols = actualizar_score(score,0,mejor_arbol_score,cols)

    
    #Printeo resultados
    print("Mejor score hasta ahora: "+str(mejor_arbol_score))
    print("Score del arbol: "+str(x)+" es "+str(arbol_score))
    print("Las columnas fueron :" +str(cols))
    print('#--------------------------------------------------------')
    
    #Opcional por si se quiere usar un promedio de todos los resultados para predecir
    prediccion=pd.merge(labels_predict,data,on='person',how='inner')
    prediccion = prediccion.fillna(0)
    x_final=prediccion[features]
    y_final=c.predict(x_final)
    nombre_columna = 'label' +str(x)
    prediccion[nombre_columna]=y_final
    prediccion = prediccion[['person', nombre_columna]]
    predicciones = pd.merge(prediccion,predicciones,on='person',how = 'inner')
    
#Printeo mejor arbol individual    
print('#--------------------------------------------------------')
print("Mejores columnas: "+str(columnas_arbol_mejor_score))
print("Mejor score: "+str(mejor_arbol_score))
print('#--------------------------------------------------------')

Arbol : 0
Mejor score hasta ahora: 73.80952380952381
Score del arbol: 0 es 73.80952380952381
Las columnas fueron :['person', 'brand listing', 'checkout', 'generic listing', 'lead', 'retornos', 'cantidad mayusculas', 'max_session_time']
#--------------------------------------------------------
Arbol : 1
Mejor score hasta ahora: 78.06122448979592
Score del arbol: 1 es 78.06122448979592
Las columnas fueron :['person', 'Email', 'Organic', 'Social', 'Unknown', 'checkout', 'conversion', 'generic listing', 'lead', 'search engine hit', 'staticpage', 'promedio dias', 'retornos', 'longitud busqueda', 'cantidad mayusculas', 'coincide', 'sessions', 'mean_time_by_session', 'mean_events_by_session', 'max_events_by_session', 'days_since_last_session', 'promedio de TFIdf', 'different_products_views']
#--------------------------------------------------------
Arbol : 2
Mejor score hasta ahora: 78.06122448979592
Score del arbol: 2 es 75.17006802721087
Las columnas fueron :['person', 'Email', 'Organic', '

In [10]:
#entrenamiento = entrenamiento.drop(columns = {'label_x'})
entrenamiento.rename(columns = {'label_y':'label'}, inplace = True)
entrenamiento.head()

,person,label_x,label,pred0,pred1,pred2,pred3,pred4,pred5,pred6,...,pred90,pred91,pred92,pred93,pred94,pred95,pred96,pred97,pred98,pred99
0,3025dc8a,0,0,1,1,1,1,1,1,1,...,1,1,1,0,1,1,1,1,1,1
1,66ab1341,0,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,6cf2d02a,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,116851fe,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ca9e125e,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,1,0,0


In [11]:
predicciones.head()

,person,label99,label98,label97,label96,label95,label94,label93,label92,label91,...,label9,label8,label7,label6,label5,label4,label3,label2,label1,label0
0,4886f805,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0297fc1e,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1
2,2d681dd8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,cccea85e,1,1,1,1,1,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1
4,4c8a8b93,1,1,1,1,1,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1


In [12]:
for x in range(50):
    nombre = 'label'+str(x)
    nuevo_nombre = 'pred'+str(x)
    predicciones.rename(columns = {nombre:nuevo_nombre}, inplace = True)
predicciones.head()

,person,label99,label98,label97,label96,label95,label94,label93,label92,label91,...,pred9,pred8,pred7,pred6,pred5,pred4,pred3,pred2,pred1,pred0
0,4886f805,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0297fc1e,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1
2,2d681dd8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,cccea85e,1,1,1,1,1,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1
4,4c8a8b93,1,1,1,1,1,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1


In [13]:
entrenamiento.to_csv('entrenamiento.csv',index=False)
predicciones.to_csv('predkaggle.csv',index=False)